# Batch Reactor Simulation - Comprehensive Examples

This notebook demonstrates the full capabilities of the batch reactor simulation system.

## Contents
1. **Setup & Imports**
2. **Example 1: E. coli in M9 Minimal Medium**
3. **Example 2: CHO Cell Culture for mAb Production**
4. **Example 3: Methanotroph on Methane**
5. **Example 4: Yeast (S. cerevisiae) Fermentation**
6. **Example 5: Lactic Acid Bacteria**
7. **Example 6: High-Cell-Density E. coli**
8. **Example 7: Parameter Sensitivity Analysis**
9. **Example 8: Multi-Scenario Comparison**
10. **Example 9: Integration with Medium Analyzer**
11. **Example 10: Fed-Batch Simulation Preview**

---

## 1. Setup & Imports

In [ ]:
# Core imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display, Markdown

# Simulation modules
from base_models import (
    CellParameters, ReactorConfig, ReactorState, 
    ProductionModel, GasComposition
)
from reactor import BatchReactor
from simulator import BatchSimulator, run_quick_simulation
from utils import (
    plot_batch_results, plot_comparison, print_summary_table,
    calculate_yields, calculate_productivities, 
    find_exponential_phase, analyze_phase
)

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("✓ All modules imported successfully!")
print("✓ Ready to simulate!")

---
## 2. Example 1: E. coli in M9 Minimal Medium

Classic aerobic growth of *E. coli* on glucose in M9 minimal medium.

### Organism Characteristics:
- **Organism:** *Escherichia coli* K-12
- **Medium:** M9 minimal medium with glucose
- **Temperature:** 37°C
- **pH:** 7.0
- **Typical μ_max:** 0.6-0.7 h⁻¹
- **Typical Yx/s:** 0.4-0.5 g/g

In [ ]:
# E. coli parameters on glucose
ecoli_params = CellParameters(
    mu_max=0.65,              # h⁻¹
    Ks=0.1,                   # g/L - half-saturation constant
    Yx_s_max=0.48,           # g/g - maximum biomass yield
    Yp_s=0.0,                # No product (just growth)
    ms=0.035,                # g/g/h - maintenance coefficient
    production_model=ProductionModel.GROWTH_ASSOCIATED,
    alpha=0.0,
    beta=0.0,
    Yx_o2=1.1,               # g biomass / g O2
    RQ=1.05                  # CO2/O2 (slightly > 1 for glucose)
)

# M9 medium configuration (typical lab scale)
m9_config = ReactorConfig(
    V_reactor=2.0,           # L - flask/small bioreactor
    V_working=1.5,           # L - 75% fill
    kLa_O2=120.0,           # h⁻¹ - good aeration
    kLa_CO2=95.0,           # h⁻¹
    T_set=37.0,             # °C
    cooling_capacity=500.0,  # W
    X0=0.05,                # g/L - low inoculum
    S0=20.0,                # g/L - glucose
    P0=0.0,
    pH0=7.0
)

# Create and run simulation
print("Simulating E. coli batch culture in M9 medium...\n")
ecoli_reactor = BatchReactor(ecoli_params, m9_config)
ecoli_sim = BatchSimulator(ecoli_reactor)
ecoli_results = ecoli_sim.simulate(t_end=24.0, dt=0.1, verbose=True)

In [ ]:
# Analyze results
print_summary_table(ecoli_results, X0=0.05, S0=20.0)

# Find exponential phase
t_start, t_end, avg_mu = find_exponential_phase(ecoli_results, mu_threshold=0.9)
print(f"\n📊 Exponential Phase Analysis:")
print(f"   Duration: {t_start:.2f} - {t_end:.2f} h ({t_end-t_start:.2f} h)")
print(f"   Average μ: {avg_mu:.4f} h⁻¹")
print(f"   Doubling time: {np.log(2)/avg_mu*60:.1f} minutes")

In [ ]:
# Visualize
plot_batch_results(ecoli_results)

### Key Observations:
- Classic diauxic growth curve
- Exponential phase during high substrate availability
- Linear substrate consumption during exponential phase
- Heat generation peaks during exponential phase
- pH remains relatively stable (M9 is well-buffered)

---
## 3. Example 2: CHO Cell Culture for mAb Production

Chinese Hamster Ovary (CHO) cells producing monoclonal antibodies (mAbs).

### Organism Characteristics:
- **Cell Line:** CHO-K1 or CHO-DG44
- **Product:** Monoclonal antibody (IgG)
- **Medium:** Complex (DMEM-based + supplements)
- **Temperature:** 37°C
- **pH:** 7.2
- **Typical μ_max:** 0.02-0.04 h⁻¹ (much slower than bacteria!)
- **Typical Yx/s:** 0.1-0.2 g/g
- **Production:** Mixed model (growth + stationary phase)

In [ ]:
# CHO cell parameters
cho_params = CellParameters(
    mu_max=0.035,            # h⁻¹ - mammalian cells grow slowly
    Ks=0.5,                  # g/L - higher Ks than bacteria
    Yx_s_max=0.15,          # g/g - low yield (complex metabolism)
    Yp_s=0.05,              # g mAb / g substrate
    ms=0.005,               # g/g/h - low maintenance
    production_model=ProductionModel.MIXED,
    alpha=0.8,               # Growth-associated production
    beta=0.015,              # Non-growth-associated production (key for mAbs!)
    Yx_o2=0.8,              # g biomass / g O2
    RQ=0.95,                # Respiratory quotient
    product_pKa=None        # mAb is neutral protein
)

# CHO culture configuration (production scale)
cho_config = ReactorConfig(
    V_reactor=10.0,          # L - benchtop bioreactor
    V_working=7.0,           # L
    kLa_O2=80.0,            # h⁻¹ - mammalian cells need less O2
    kLa_CO2=65.0,           # h⁻¹
    T_set=37.0,             # °C
    cooling_capacity=1000.0, # W
    X0=0.3,                 # g/L - higher inoculum
    S0=25.0,                # g/L - glucose (+ glutamine in real medium)
    P0=0.0,                 # g/L - no initial mAb
    pH0=7.2
)

# Run simulation (longer time for slow growth)
print("Simulating CHO cell culture for mAb production...\n")
cho_reactor = BatchReactor(cho_params, cho_config)
cho_sim = BatchSimulator(cho_reactor)
cho_results = cho_sim.simulate(t_end=168.0, dt=0.5, verbose=True)  # 7 days

In [ ]:
# Analyze CHO culture
print_summary_table(cho_results, X0=0.3, S0=25.0)

# Calculate specific productivities
prod = calculate_productivities(cho_results, X0=0.3, S0=25.0)
print(f"\n💊 mAb Production Metrics:")
print(f"   Final titer: {cho_results.P[-1]:.3f} g/L")
print(f"   Volumetric productivity: {prod['Qp']*1000:.2f} mg/L/h")
print(f"   Specific productivity: {prod['qp']*1000:.3f} mg/g-cell/h")

# Analyze growth and production phases
growth_phase = analyze_phase(cho_results, t_start=0, t_end=72)
production_phase = analyze_phase(cho_results, t_start=72, t_end=168)

print(f"\n📈 Phase Analysis:")
print(f"   Growth phase (0-72h):")
print(f"      Biomass increase: {growth_phase['biomass_change']:.2f} g/L")
print(f"      Product formed: {growth_phase['product_formed']:.3f} g/L")
print(f"   Production phase (72-168h):")
print(f"      Biomass increase: {production_phase['biomass_change']:.2f} g/L")
print(f"      Product formed: {production_phase['product_formed']:.3f} g/L")

In [ ]:
# Visualize CHO culture
plot_batch_results(cho_results)

### Key Observations:
- Much slower growth than bacteria (168h vs 24h)
- Product continues accumulating after growth slows (non-growth-associated β)
- Lower OUR due to slower metabolism
- Temperature control is critical for mammalian cells
- In practice: fed-batch preferred for extended production phase

---
## 4. Example 3: Methanotroph on Methane

Methylococcus capsulatus or similar methanotroph growing on methane (CH₄) as sole carbon source.

### Organism Characteristics:
- **Organism:** *Methylococcus capsulatus* (Bath)
- **Substrate:** Methane (CH₄) - gaseous!
- **Product:** Single-cell protein (SCP) or methanol
- **Temperature:** 45°C (thermophile)
- **pH:** 6.8-7.0
- **Key feature:** High O₂ demand (oxygenase enzyme)
- **Typical μ_max:** 0.15-0.25 h⁻¹
- **Typical Yx/s:** 0.5-0.6 g/g (on CH₄-C basis)

In [ ]:
# Methanotroph parameters
methano_params = CellParameters(
    mu_max=0.20,             # h⁻¹
    Ks=0.015,                # g/L - CH4 has low solubility
    Yx_s_max=0.55,          # g biomass / g CH4
    Yp_s=0.0,               # No product in this example (just biomass)
    ms=0.02,                # g/g/h
    production_model=ProductionModel.GROWTH_ASSOCIATED,
    alpha=0.0,
    beta=0.0,
    Yx_o2=0.5,              # LOW! High O2 demand for CH4 oxidation
    RQ=0.7,                 # CO2/O2 < 1 (less CO2 per O2 than glucose)
)

# Methanotroph culture with CH4-enriched gas feed
methane_gas = GasComposition(
    O2=0.21,                # 21% O2
    CO2=0.01,               # 1% CO2
    N2=0.58,                # 58% N2
    CH4=0.20,               # 20% CH4 (balance)
    H2=0.0
)

methano_config = ReactorConfig(
    V_reactor=5.0,           # L
    V_working=3.0,           # L
    inlet_gas=methane_gas,
    gas_flow_rate=1.5,       # VVM - high for gas substrate
    kLa_O2=200.0,           # h⁻¹ - VERY HIGH (O2 critical)
    kLa_CO2=180.0,          # h⁻¹
    T_set=45.0,             # °C - thermophile!
    cooling_capacity=800.0,
    X0=0.1,                 # g/L
    S0=0.5,                 # g/L - "dissolved" CH4 (simplified)
    P0=0.0,
    pH0=6.8
)

print("Simulating methanotroph culture on methane...\n")
print("⚠️  Note: This is a simplified model. Real CH4 fermentation requires")
print("    detailed gas-liquid mass transfer and CH4 solubility modeling.\n")

methano_reactor = BatchReactor(methano_params, methano_config)
methano_sim = BatchSimulator(methano_reactor)
methano_results = methano_sim.simulate(t_end=36.0, dt=0.2, verbose=True)

In [ ]:
# Analyze methanotroph culture
print_summary_table(methano_results, X0=0.1, S0=0.5)

# High oxygen demand analysis
print(f"\n🔥 Oxygen Demand Analysis:")
print(f"   Peak OUR: {np.max(methano_results.OUR):.2f} mmol/L/h")
print(f"   Average OUR: {np.mean(methano_results.OUR):.2f} mmol/L/h")
print(f"   Total O2 consumed: {np.trapz(methano_results.OUR, methano_results.time)/1000:.2f} mol")
print(f"   Biomass/O2 yield: {methano_params.Yx_o2:.2f} g/g (very low = O2 intensive!)")

# RQ analysis
RQ_actual = methano_results.CER / (methano_results.OUR + 1e-10)
print(f"\n📊 Respiratory Quotient:")
print(f"   Theoretical RQ: {methano_params.RQ:.2f}")
print(f"   Average RQ: {np.mean(RQ_actual[RQ_actual<2]):.2f}")
print(f"   (RQ < 1 indicates less CO2 produced than O2 consumed)")

In [ ]:
# Visualize
plot_batch_results(methano_results)

### Key Observations:
- **Very high O₂ demand** - methanotrophs need ~2 mol O₂ per mol CH₄
- RQ < 1 because methane oxidation consumes O₂ without equivalent CO₂ release
- High kLa required to avoid O₂ limitation
- Heat generation significant due to oxidative metabolism
- Temperature control critical (thermophile at 45°C)
- In practice: O₂ is often the limiting factor, not CH₄

---
## 5. Example 4: Yeast (S. cerevisiae) Fermentation

*Saccharomyces cerevisiae* growing on glucose - comparing aerobic vs. anaerobic.

### Organism Characteristics:
- **Organism:** *Saccharomyces cerevisiae* (baker's yeast)
- **Substrate:** Glucose
- **Product:** Ethanol (under anaerobic/semi-aerobic)
- **Temperature:** 30°C
- **pH:** 4.5-5.5 (acidic)
- **Typical μ_max:** 0.4-0.5 h⁻¹ (aerobic), 0.2-0.3 h⁻¹ (anaerobic)
- **Crabtree effect:** Produces ethanol even with O₂ at high glucose

In [ ]:
# Yeast parameters - aerobic respiration
yeast_aerobic_params = CellParameters(
    mu_max=0.45,             # h⁻¹
    Ks=0.2,                  # g/L
    Yx_s_max=0.50,          # g/g - high yield (respiration)
    Yp_s=0.05,              # g ethanol/g glucose (minor fermentation)
    ms=0.015,               # g/g/h
    production_model=ProductionModel.MIXED,
    alpha=0.1,               # Small amount of growth-associated ethanol
    beta=0.005,              # Minimal non-growth-associated
    Yx_o2=1.0,              # g biomass / g O2
    RQ=1.0,                 # Fully respiratory
    product_pKa=None        # Ethanol doesn't affect pH much
)

# Yeast parameters - fermentative (low O2)
yeast_ferment_params = CellParameters(
    mu_max=0.25,             # h⁻¹ - slower without O2
    Ks=0.2,                  # g/L
    Yx_s_max=0.10,          # g/g - LOW yield (fermentation)
    Yp_s=0.46,              # g ethanol/g glucose (theoretical max ~0.51)
    ms=0.01,                # g/g/h - lower maintenance
    production_model=ProductionModel.GROWTH_ASSOCIATED,
    alpha=2.0,               # High ethanol production
    beta=0.0,
    Yx_o2=0.0,              # Essentially anaerobic
    RQ=10.0,                # Lots of CO2, minimal O2 (fermentation)
    product_pKa=None
)

# Aerobic config
yeast_aero_config = ReactorConfig(
    V_reactor=3.0,
    V_working=2.0,
    kLa_O2=150.0,           # Good aeration
    kLa_CO2=120.0,
    T_set=30.0,             # °C - yeast prefers cooler
    X0=0.1,
    S0=50.0,                # g/L - high glucose
    P0=0.0,
    pH0=5.0                 # Acidic for yeast
)

# Anaerobic config
yeast_anaero_config = ReactorConfig(
    V_reactor=3.0,
    V_working=2.0,
    kLa_O2=10.0,            # Minimal aeration
    kLa_CO2=120.0,          # CO2 stripping important
    T_set=30.0,
    X0=0.1,
    S0=50.0,
    P0=0.0,
    pH0=5.0
)

print("Simulating yeast fermentation...\n")
print("Running AEROBIC condition (high O2)...")
yeast_aero_reactor = BatchReactor(yeast_aerobic_params, yeast_aero_config)
yeast_aero_sim = BatchSimulator(yeast_aero_reactor)
yeast_aero_results = yeast_aero_sim.simulate(t_end=30.0, dt=0.2, verbose=False)
print("✓ Aerobic simulation complete\n")

print("Running FERMENTATIVE condition (low O2)...")
yeast_anaero_reactor = BatchReactor(yeast_ferment_params, yeast_anaero_config)
yeast_anaero_sim = BatchSimulator(yeast_anaero_reactor)
yeast_anaero_results = yeast_anaero_sim.simulate(t_end=30.0, dt=0.2, verbose=False)
print("✓ Fermentative simulation complete")

In [ ]:
# Compare aerobic vs fermentative
print("\n" + "="*70)
print("AEROBIC vs FERMENTATIVE COMPARISON")
print("="*70)

aero_yields = calculate_yields(yeast_aero_results, 0.1, 50.0)
ferm_yields = calculate_yields(yeast_anaero_results, 0.1, 50.0)

print("\n📊 AEROBIC (Respiration):")
print(f"   Final biomass: {aero_yields['final_biomass']:.2f} g/L")
print(f"   Final ethanol: {aero_yields['final_product']:.2f} g/L")
print(f"   Yx/s: {aero_yields['Yx_s']:.3f} g/g (HIGH yield)")
print(f"   Yp/s: {aero_yields['Yp_s']:.3f} g/g (low ethanol)")
print(f"   Growth rate: Fast (μ_max = 0.45 h⁻¹)")

print("\n📊 FERMENTATIVE (Anaerobic):")
print(f"   Final biomass: {ferm_yields['final_biomass']:.2f} g/L")
print(f"   Final ethanol: {ferm_yields['final_product']:.2f} g/L")
print(f"   Yx/s: {ferm_yields['Yx_s']:.3f} g/g (LOW yield)")
print(f"   Yp/s: {ferm_yields['Yp_s']:.3f} g/g (HIGH ethanol!)")
print(f"   Growth rate: Slower (μ_max = 0.25 h⁻¹)")

print("\n🔬 Key Insight: Pasteur Effect")
print("   With O2: More biomass, less ethanol, faster growth")
print("   Without O2: Less biomass, MORE ethanol, slower growth")
print("   → Beer/wine production uses limited O2 to maximize ethanol!")

In [ ]:
# Visual comparison
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Biomass
axes[0,0].plot(yeast_aero_results.time, yeast_aero_results.X, 'b-', linewidth=2, label='Aerobic')
axes[0,0].plot(yeast_anaero_results.time, yeast_anaero_results.X, 'r--', linewidth=2, label='Fermentative')
axes[0,0].set_xlabel('Time (h)')
axes[0,0].set_ylabel('Biomass (g/L)')
axes[0,0].set_title('Biomass Growth')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Substrate
axes[0,1].plot(yeast_aero_results.time, yeast_aero_results.S, 'b-', linewidth=2, label='Aerobic')
axes[0,1].plot(yeast_anaero_results.time, yeast_anaero_results.S, 'r--', linewidth=2, label='Fermentative')
axes[0,1].set_xlabel('Time (h)')
axes[0,1].set_ylabel('Glucose (g/L)')
axes[0,1].set_title('Substrate Consumption')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# Ethanol
axes[0,2].plot(yeast_aero_results.time, yeast_aero_results.P, 'b-', linewidth=2, label='Aerobic')
axes[0,2].plot(yeast_anaero_results.time, yeast_anaero_results.P, 'r--', linewidth=2, label='Fermentative')
axes[0,2].set_xlabel('Time (h)')
axes[0,2].set_ylabel('Ethanol (g/L)')
axes[0,2].set_title('Ethanol Production')
axes[0,2].legend()
axes[0,2].grid(True, alpha=0.3)

# Growth rate
axes[1,0].plot(yeast_aero_results.time, yeast_aero_results.mu, 'b-', linewidth=2, label='Aerobic')
axes[1,0].plot(yeast_anaero_results.time, yeast_anaero_results.mu, 'r--', linewidth=2, label='Fermentative')
axes[1,0].set_xlabel('Time (h)')
axes[1,0].set_ylabel('μ (h⁻¹)')
axes[1,0].set_title('Specific Growth Rate')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# OUR
axes[1,1].plot(yeast_aero_results.time, yeast_aero_results.OUR, 'b-', linewidth=2, label='Aerobic')
axes[1,1].plot(yeast_anaero_results.time, yeast_anaero_results.OUR, 'r--', linewidth=2, label='Fermentative')
axes[1,1].set_xlabel('Time (h)')
axes[1,1].set_ylabel('OUR (mmol/L/h)')
axes[1,1].set_title('Oxygen Uptake Rate')
axes[1,1].legend()
axes[1,1].grid(True, alpha=0.3)

# Yield comparison (bar chart)
yields_comparison = pd.DataFrame({
    'Aerobic': [aero_yields['Yx_s'], aero_yields['Yp_s']],
    'Fermentative': [ferm_yields['Yx_s'], ferm_yields['Yp_s']]
}, index=['Yx/s (biomass)', 'Yp/s (ethanol)'])

yields_comparison.plot(kind='bar', ax=axes[1,2], rot=45)
axes[1,2].set_ylabel('Yield (g/g)')
axes[1,2].set_title('Yield Comparison')
axes[1,2].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n💡 This demonstrates the Pasteur Effect:")
print("   Aerobic metabolism → efficient biomass production")
print("   Fermentative metabolism → inefficient but produces ethanol")

---
## 6. Example 5: Lactic Acid Bacteria

*Lactobacillus* or *Lactococcus* producing lactic acid from glucose.

### Key Features:
- Fermentative (no O₂ requirement)
- Produces lactic acid → pH drops significantly
- Product inhibition occurs
- Used in dairy industry (yogurt, cheese)

In [ ]:
# Lactic acid bacteria parameters
lab_params = CellParameters(
    mu_max=0.5,              # h⁻¹
    Ks=0.3,                  # g/L
    Yx_s_max=0.12,          # g/g - low (most goes to lactate)
    Yp_s=0.85,              # g lactate / g glucose (near theoretical)
    ms=0.008,               # g/g/h
    production_model=ProductionModel.GROWTH_ASSOCIATED,
    alpha=7.0,               # High lactate production
    beta=0.0,
    Yx_o2=0.0,              # Anaerobic
    RQ=0.0,                 # No O2 consumption
    product_pKa=3.86        # Lactic acid pKa
)

# LAB fermentation config
lab_config = ReactorConfig(
    V_reactor=5.0,
    V_working=3.5,
    kLa_O2=5.0,             # Very low (anaerobic)
    kLa_CO2=50.0,
    T_set=37.0,
    X0=0.2,                 # Higher inoculum
    S0=80.0,                # High glucose for lactate production
    P0=0.0,
    pH0=6.5                 # Starts slightly acidic
)

print("Simulating lactic acid fermentation...\n")
lab_reactor = BatchReactor(lab_params, lab_config)
lab_sim = BatchSimulator(lab_reactor)
lab_results = lab_sim.simulate(t_end=24.0, dt=0.2, verbose=True)

In [ ]:
# Analyze lactate production
print_summary_table(lab_results, X0=0.2, S0=80.0)

print(f"\n🥛 Lactic Acid Production:")
print(f"   Final lactate titer: {lab_results.P[-1]:.1f} g/L")
print(f"   Final pH: {lab_results.pH[-1]:.2f} (started at {lab_config.pH0})")
print(f"   pH drop: {lab_config.pH0 - lab_results.pH[-1]:.2f} units")
print(f"   Conversion: {lab_results.P[-1]/lab_config.S0*100:.1f}% of glucose to lactate")

# Find when pH drops below 5.0 (typical product inhibition)
pH_limit_idx = np.where(lab_results.pH < 5.0)[0]
if len(pH_limit_idx) > 0:
    t_pH_limit = lab_results.time[pH_limit_idx[0]]
    print(f"\n⚠️  pH dropped below 5.0 at t = {t_pH_limit:.1f} h")
    print(f"   Growth likely inhibited after this point")
    print(f"   → Industrial process would use pH control (NaOH addition)")

In [ ]:
# Visualize with focus on pH
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Concentrations
ax = axes[0,0]
ax.plot(lab_results.time, lab_results.X, 'b-', linewidth=2, label='Biomass')
ax.plot(lab_results.time, lab_results.S, 'r--', linewidth=2, label='Glucose')
ax.plot(lab_results.time, lab_results.P, 'g-.', linewidth=2, label='Lactate')
ax.set_xlabel('Time (h)')
ax.set_ylabel('Concentration (g/L)')
ax.set_title('Fermentation Profile')
ax.legend()
ax.grid(True, alpha=0.3)

# pH profile
ax = axes[0,1]
ax.plot(lab_results.time, lab_results.pH, 'purple', linewidth=2)
ax.axhline(y=5.0, color='r', linestyle='--', label='Inhibition threshold')
ax.set_xlabel('Time (h)')
ax.set_ylabel('pH')
ax.set_title('pH Drop Due to Lactic Acid')
ax.legend()
ax.grid(True, alpha=0.3)

# Growth rate vs pH
ax = axes[1,0]
ax.scatter(lab_results.pH, lab_results.mu, alpha=0.5, s=20)
ax.set_xlabel('pH')
ax.set_ylabel('μ (h⁻¹)')
ax.set_title('Growth Rate vs pH')
ax.grid(True, alpha=0.3)
ax.invert_xaxis()  # pH decreases with time

# Lactate vs glucose
ax = axes[1,1]
ax.plot(lab_results.S, lab_results.P, 'g-', linewidth=2)
ax.scatter([lab_results.S[0]], [lab_results.P[0]], c='blue', s=100, 
           marker='o', label='Start', zorder=5)
ax.scatter([lab_results.S[-1]], [lab_results.P[-1]], c='red', s=100, 
           marker='s', label='End', zorder=5)
ax.set_xlabel('Glucose (g/L)')
ax.set_ylabel('Lactate (g/L)')
ax.set_title('Product Formation')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 7. Example 6: High-Cell-Density E. coli

Fed-batch-like conditions with high biomass and substrate feeding.

### Scenario:
- Start with high initial biomass
- Use multiple substrate additions (simulated as high S0)
- Target: >50 g/L biomass

In [ ]:
# High-density E. coli parameters (similar to fed-batch)
hcd_params = CellParameters(
    mu_max=0.55,             # Slightly lower at high density
    Ks=0.15,                 # g/L
    Yx_s_max=0.45,          # g/g
    Yp_s=0.0,
    ms=0.04,                # Higher maintenance at high density
    production_model=ProductionModel.GROWTH_ASSOCIATED,
    alpha=0.0,
    beta=0.0,
    Yx_o2=1.0,
    RQ=1.05
)

hcd_config = ReactorConfig(
    V_reactor=10.0,          # Larger bioreactor
    V_working=7.0,
    kLa_O2=250.0,           # Very high for O2 demand
    kLa_CO2=200.0,
    T_set=37.0,
    cooling_capacity=2000.0, # High cooling needed
    X0=5.0,                 # HIGH initial biomass
    S0=100.0,               # HIGH substrate (simulating fed-batch)
    P0=0.0,
    pH0=7.0
)

print("Simulating high-cell-density E. coli culture...\n")
print("⚠️  Note: True HCD requires fed-batch with controlled feeding.")
print("    This is a simplified batch with high initial conditions.\n")

hcd_reactor = BatchReactor(hcd_params, hcd_config)
hcd_sim = BatchSimulator(hcd_reactor)
hcd_results = hcd_sim.simulate(t_end=18.0, dt=0.1, verbose=True)

In [ ]:
# Analyze HCD culture
print_summary_table(hcd_results, X0=5.0, S0=100.0)

print(f"\n🚀 High-Cell-Density Metrics:")
print(f"   Peak biomass: {np.max(hcd_results.X):.1f} g/L")
print(f"   Final biomass: {hcd_results.X[-1]:.1f} g/L")
print(f"   Peak OUR: {np.max(hcd_results.OUR):.1f} mmol/L/h")
print(f"   Peak heat generation: {np.max(hcd_results.Q_gen):.1f} W")
print(f"   Total heat: {np.trapz(hcd_results.Q_gen, hcd_results.time)*3.6:.1f} kJ")

print(f"\n⚡ Process Intensification:")
print(f"   Volumetric productivity: {hcd_results.X[-1]/hcd_results.time[-1]:.2f} g/L/h")
print(f"   Space-time yield: {(hcd_results.X[-1]-5.0)/hcd_results.time[-1]:.2f} g/L/h")
print(f"   O2 demand: VERY HIGH - requires pure O2 or high agitation")
print(f"   Cooling demand: {np.mean(hcd_results.Q_gen):.0f} W average")

In [ ]:
# Visualize HCD challenges
plot_batch_results(hcd_results)

---
## 8. Example 7: Parameter Sensitivity Analysis

Systematic exploration of how parameters affect the outcome.

In [ ]:
print("Running parameter sensitivity analysis...\n")

# Test different mu_max values
mu_values = [0.3, 0.5, 0.7, 0.9]
mu_results = []

for mu in mu_values:
    result = run_quick_simulation(mu_max=mu, Ks=0.1, Yx_s=0.5, 
                                   ms=0.03, S0=20.0, X0=0.1, t_end=24.0)
    mu_results.append(result)
    print(f"✓ μ_max = {mu:.1f} h⁻¹: Final X = {result.X[-1]:.2f} g/L")

# Test different Yx_s values
print("\n")
yield_values = [0.3, 0.4, 0.5, 0.6]
yield_results = []

for y in yield_values:
    result = run_quick_simulation(mu_max=0.7, Ks=0.1, Yx_s=y, 
                                   ms=0.03, S0=20.0, X0=0.1, t_end=24.0)
    yield_results.append(result)
    print(f"✓ Yx/s = {y:.1f} g/g: Final X = {result.X[-1]:.2f} g/L")

# Test different maintenance values
print("\n")
ms_values = [0.0, 0.01, 0.03, 0.05]
ms_results = []

for ms in ms_values:
    result = run_quick_simulation(mu_max=0.7, Ks=0.1, Yx_s=0.5, 
                                   ms=ms, S0=20.0, X0=0.1, t_end=24.0)
    ms_results.append(result)
    print(f"✓ ms = {ms:.2f} g/g/h: Final X = {result.X[-1]:.2f} g/L")

In [ ]:
# Visualize sensitivity
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# mu_max sensitivity - biomass
ax = axes[0,0]
for i, (result, mu) in enumerate(zip(mu_results, mu_values)):
    ax.plot(result.time, result.X, linewidth=2, label=f'μ_max={mu:.1f}')
ax.set_xlabel('Time (h)')
ax.set_ylabel('Biomass (g/L)')
ax.set_title('Effect of μ_max on Growth')
ax.legend()
ax.grid(True, alpha=0.3)

# mu_max sensitivity - growth rate
ax = axes[0,1]
for i, (result, mu) in enumerate(zip(mu_results, mu_values)):
    ax.plot(result.time, result.mu, linewidth=2, label=f'μ_max={mu:.1f}')
ax.set_xlabel('Time (h)')
ax.set_ylabel('μ (h⁻¹)')
ax.set_title('Growth Rate Profiles')
ax.legend()
ax.grid(True, alpha=0.3)

# mu_max - final biomass
ax = axes[0,2]
final_X = [r.X[-1] for r in mu_results]
ax.bar(range(len(mu_values)), final_X, tick_label=[f'{m:.1f}' for m in mu_values])
ax.set_xlabel('μ_max (h⁻¹)')
ax.set_ylabel('Final Biomass (g/L)')
ax.set_title('Final Biomass vs μ_max')
ax.grid(True, alpha=0.3, axis='y')

# Yx_s sensitivity
ax = axes[1,0]
for i, (result, y) in enumerate(zip(yield_results, yield_values)):
    ax.plot(result.time, result.X, linewidth=2, label=f'Yx/s={y:.1f}')
ax.set_xlabel('Time (h)')
ax.set_ylabel('Biomass (g/L)')
ax.set_title('Effect of Yield on Growth')
ax.legend()
ax.grid(True, alpha=0.3)

# ms sensitivity
ax = axes[1,1]
for i, (result, ms) in enumerate(zip(ms_results, ms_values)):
    ax.plot(result.time, result.X, linewidth=2, label=f'ms={ms:.2f}')
ax.set_xlabel('Time (h)')
ax.set_ylabel('Biomass (g/L)')
ax.set_title('Effect of Maintenance on Growth')
ax.legend()
ax.grid(True, alpha=0.3)

# Summary: final biomass vs parameters
ax = axes[1,2]
param_names = ['μ_0.3', 'μ_0.7', 'Y_0.3', 'Y_0.6', 'ms_0.0', 'ms_0.05']
param_values = [
    mu_results[0].X[-1], mu_results[-1].X[-1],
    yield_results[0].X[-1], yield_results[-1].X[-1],
    ms_results[0].X[-1], ms_results[-1].X[-1]
]
colors = ['blue', 'blue', 'green', 'green', 'red', 'red']
ax.barh(range(len(param_names)), param_values, color=colors, alpha=0.7)
ax.set_yticks(range(len(param_names)))
ax.set_yticklabels(param_names)
ax.set_xlabel('Final Biomass (g/L)')
ax.set_title('Parameter Sensitivity Summary')
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("\n📊 Key Insights:")
print("   • μ_max: Affects growth rate and time to stationary phase")
print("   • Yx/s: Directly determines final biomass from substrate")
print("   • ms: Higher maintenance reduces overall yield and final biomass")

---
## 9. Example 8: Multi-Scenario Comparison

Compare all organisms side-by-side.

In [ ]:
# Create summary comparison table
scenarios = [
    ('E. coli (M9)', ecoli_results, 0.05, 20.0),
    ('CHO cells', cho_results, 0.3, 25.0),
    ('Methanotroph', methano_results, 0.1, 0.5),
    ('Yeast (aerobic)', yeast_aero_results, 0.1, 50.0),
    ('LAB (lactate)', lab_results, 0.2, 80.0),
]

comparison_data = []
for name, result, X0, S0 in scenarios:
    yields = calculate_yields(result, X0, S0)
    prod = calculate_productivities(result, X0, S0)
    
    comparison_data.append({
        'Organism': name,
        'Duration (h)': result.time[-1],
        'Final X (g/L)': result.X[-1],
        'Final P (g/L)': result.P[-1],
        'Yx/s (g/g)': yields['Yx_s'],
        'Yp/s (g/g)': yields['Yp_s'],
        'Max μ (1/h)': np.max(result.mu),
        'Qx (g/L/h)': prod['Qx'],
        'Avg OUR (mmol/L/h)': np.mean(result.OUR)
    })

df_comparison = pd.DataFrame(comparison_data)
print("\n" + "="*100)
print("COMPREHENSIVE ORGANISM COMPARISON")
print("="*100)
display(df_comparison.round(3))

In [ ]:
# Visual comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Growth rates
ax = axes[0,0]
x_pos = np.arange(len(df_comparison))
ax.bar(x_pos, df_comparison['Max μ (1/h)'], color='steelblue', alpha=0.7)
ax.set_xticks(x_pos)
ax.set_xticklabels(df_comparison['Organism'], rotation=45, ha='right')
ax.set_ylabel('μ_max (h⁻¹)')
ax.set_title('Maximum Growth Rates')
ax.grid(True, alpha=0.3, axis='y')

# Yields
ax = axes[0,1]
x_pos = np.arange(len(df_comparison))
width = 0.35
ax.bar(x_pos - width/2, df_comparison['Yx/s (g/g)'], width, 
       label='Biomass yield', alpha=0.7)
ax.bar(x_pos + width/2, df_comparison['Yp/s (g/g)'], width, 
       label='Product yield', alpha=0.7)
ax.set_xticks(x_pos)
ax.set_xticklabels(df_comparison['Organism'], rotation=45, ha='right')
ax.set_ylabel('Yield (g/g)')
ax.set_title('Biomass and Product Yields')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Productivities
ax = axes[1,0]
ax.bar(x_pos, df_comparison['Qx (g/L/h)'], color='green', alpha=0.7)
ax.set_xticks(x_pos)
ax.set_xticklabels(df_comparison['Organism'], rotation=45, ha='right')
ax.set_ylabel('Qx (g/L/h)')
ax.set_title('Biomass Productivities')
ax.grid(True, alpha=0.3, axis='y')

# OUR comparison
ax = axes[1,1]
ax.bar(x_pos, df_comparison['Avg OUR (mmol/L/h)'], color='red', alpha=0.7)
ax.set_xticks(x_pos)
ax.set_xticklabels(df_comparison['Organism'], rotation=45, ha='right')
ax.set_ylabel('OUR (mmol/L/h)')
ax.set_title('Average Oxygen Uptake Rates')
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

---
## 10. Example 9: Integration with Medium Analyzer

Use the `medium_analyzer` to create recipes and run fermentations.

In [ ]:
# Try to import medium_analyzer
try:
    from medium_analyzer import ComponentLibrary, Recipe, Component
    has_medium_analyzer = True
    print("✓ Medium analyzer available!\n")
except ImportError:
    has_medium_analyzer = False
    print("⚠️  Medium analyzer not found. Skipping this example.\n")
    print("   To use: ensure medium_analyzer.py is in the same directory.")

if has_medium_analyzer:
    # Create M9 medium using medium analyzer
    lib = ComponentLibrary()
    
    m9_recipe = Recipe("M9 Medium", "Standard E. coli minimal medium")
    m9_recipe.add_component(lib.get_component("Glucose"), 20.0, "g/L")
    m9_recipe.add_component(lib.get_component("Ammonium Sulfate"), 1.0, "g/L")
    m9_recipe.add_component(lib.get_component("Potassium Phosphate Monobasic"), 3.0, "g/L")
    m9_recipe.add_component(lib.get_component("Sodium Chloride"), 0.5, "g/L")
    m9_recipe.add_component(lib.get_component("Magnesium Sulfate Heptahydrate"), 0.24, "g/L")
    
    # Analyze medium composition
    m9_analysis = m9_recipe.analyze()
    print("📋 M9 Medium Analysis:")
    print(f"   Total components: {len(m9_recipe.components)}")
    print(f"   C/N ratio: {m9_analysis.get_cn_ratio():.2f}")
    print(f"   Total carbon: {m9_analysis.get_elemental_composition()['C']:.2f} g/L")
    print(f"   Total nitrogen: {m9_analysis.get_elemental_composition()['N']:.2f} g/L")
    
    # Extract glucose for fermentation
    glucose_conc = next((conc for comp, conc in m9_analysis.composition 
                        if 'glucose' in comp.name.lower()), 20.0)
    
    print(f"\n🔬 Running fermentation with extracted parameters:")
    print(f"   Glucose concentration: {glucose_conc:.1f} g/L")
    
    # Run fermentation
    integrated_results = run_quick_simulation(
        mu_max=0.65,
        Ks=0.1,
        Yx_s=0.48,
        ms=0.035,
        S0=glucose_conc,
        X0=0.05,
        t_end=24.0
    )
    
    print(f"\n✓ Fermentation completed!")
    print(f"   Final biomass: {integrated_results.X[-1]:.2f} g/L")
    print(f"   Biomass yield: {(integrated_results.X[-1]-0.05)/glucose_conc:.3f} g/g")
    
    # Plot
    plot_batch_results(integrated_results)

---
## 11. Example 10: Advanced Topics Preview

### Topics for Future Development:
1. **Fed-batch operations** with controlled feeding
2. **pH control** with base addition
3. **Multi-substrate systems** (glucose + glycerol)
4. **Dissolved oxygen control** (DOstat)
5. **Parameter estimation** from experimental data
6. **Model predictive control** for optimization

In [ ]:
print("\n" + "="*70)
print("🎓 BATCH REACTOR EXAMPLES COMPLETE!")
print("="*70)
print("\nYou've explored:")
print("  ✓ E. coli in M9 minimal medium")
print("  ✓ CHO cell culture for mAb production")
print("  ✓ Methanotroph on methane (gaseous substrate)")
print("  ✓ Yeast fermentation (aerobic vs anaerobic)")
print("  ✓ Lactic acid bacteria (pH effects)")
print("  ✓ High-cell-density E. coli")
print("  ✓ Parameter sensitivity analysis")
print("  ✓ Multi-organism comparison")
print("  ✓ Integration with medium analyzer")
print("\n📚 Next steps:")
print("  • Modify parameters for your specific organism")
print("  • Add new production models")
print("  • Implement fed-batch control strategies")
print("  • Validate against experimental data")
print("\n💡 Remember: All models are approximations!")
print("   Always validate against experimental results.")
print("="*70)